# 1. Library & Seed Setting

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import requests
import joblib
import pickle

from math import radians, sin, cos, sqrt, atan2
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
from sklearn.neighbors import BallTree
from tqdm import tqdm

plt.rcParams['font.family'] = 'NanumGothic'

In [33]:
def seed_setting(seed=1004) :
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_setting()

# 2. Dataset Loading & Base Preprocessing

맨 처음 훈련을 한 뒤 특성 중요도를 확인해보니 다음과 같았다.

| index | 컬럼명                                     |   개수 |
|:-----:|:------------------------------------------|-------:|
| 0     | 전용면적___                               | 25,408 |
| 8     | 계약_연_                                  | 24,916 |
| 9     | 아파트명                                  | 14,339 |
| 13    | 도로명                                    | 14,255 |
| 1     | 건축년도                                  |  9,804 |
| 15    | cluster                                   |  5,308 |
| 3     | 연면적                                    |  1,100 |
| 4     | 전용면적별세대현황_60_이하_               |    677 |
| 14    | 세대당_주차대수                          |    563 |
| 2     | 해제사유발생일                            |    444 |
| 5     | 전용면적별세대현황_60__85_이하_          |    437 |
| 6     | 전체세대수                                |    436 |
| 7     | 주차대수                                  |    313 |
| 11    | 복도유형                                  |     37 |
| 12    | 단지분류_아파트_주상복합등등_            |     13 |
| 10    | 등기신청일자                              |      0 |

해당 특성들에 더하여 지하철역과 버스정류장 데이터를 이용하여 새로운 특성을 추가한다.

좌표X와 좌표Y는 이용할 곳이 많기 때문에 일단 삭제하지 않기로 결정한다.

sclaer를 Robustscalcer를 사용하기 때문에 이상치에 대해서는 어느정도 대응가능하므로, 층 특성을 다시 추가한다.

또한 금리에 따라 아파트 가격의 변동성이 있다는 가정 하에 계약(월) 특성을 유지하고 금리 특성을 추가하여 target과의 관계를 파악 후 특성을 유지할지 삭제할지 결정한다.

In [34]:
train_data = pd.read_csv('train.csv')
bus_data = pd.read_csv('bus_feature.csv')
subway_data = pd.read_csv('subway_feature.csv')

/tmp/ipykernel_1376/535313857.py:1: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train.csv')


In [1]:
def Entire_Preprocessing(df) :
    # 문자열 컬럼만 찾아서 좌우 공백 제거
    df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

    # 전화번호, 팩스번호 k-홈페이지, 고용보험관리번호, k-등록일자, k-수정일자, 관리비 업로드, 단지소개기존clob 삭제
    df = df.drop(columns=['k-전화번호', 'k-팩스번호', 'k-홈페이지', '고용보험관리번호', 'k-등록일자', 'k-수정일자', '관리비 업로드', '단지소개기존clob'])

    # 본번, 부번, 시군구 삭제
    df = df.drop(columns=['본번', '부번', '시군구'])

    # 계약년월 분해
    df['계약(연)'] = df['계약년월'] // 100
    df['계약(월)'] = df['계약년월'] % 100
    df = df.drop(columns=['계약년월'])

    # 계약일 → 계약(일)
    df = df.rename(columns={"계약일" : "계약(일)"})

    # 불필요 특성 제거
    if 'target' in df.columns :
        columns_to_keep = [
        '전용면적(㎡)', '건축년도', '해제사유발생일', 'k-연면적', 'k-전용면적별세대현황(60㎡이하)',
        'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-전체세대수', '주차대수', '계약(연)',
        '좌표X', '좌표Y', '아파트명', '등기신청일자', 'k-복도유형', 'k-단지분류(아파트,주상복합등등)', '도로명', 'target'
        ]
    else :
        columns_to_keep = [
        '전용면적(㎡)', '건축년도', '해제사유발생일', 'k-연면적', 'k-전용면적별세대현황(60㎡이하)',
        'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-전체세대수', '주차대수', '계약(연)', 
        '좌표X', '좌표Y', '아파트명', '등기신청일자', 'k-복도유형', 'k-단지분류(아파트,주상복합등등)', '도로명'
        ]

    df = df[columns_to_keep]

    # 해재사유발생일 전처리
    df['해제사유발생일'] = df['해제사유발생일'].notnull().astype(int)

    # 세대당_주차대수 특성 생성
    df['세대당_주차대수'] = df.apply(
    lambda row: row['주차대수'] / row['k-전체세대수'] if pd.notnull(row['주차대수']) and pd.notnull(row['k-전체세대수']) else np.nan,
    axis=1)

    # 등기신청일자 전처리
    df['등기신청일자'] = df['등기신청일자'].notnull().astype(int)

    # 특성 이름에서 k- 빼기
    df.columns = df.columns.str.replace('k-', '')

    # 군집화
    ## 카카오 API 호출 함수
    def get_coords_kakao(address, api_key):
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {"Authorization": f"KakaoAK {api_key}"}
        params = {"query": address}
        response = requests.get(url, headers=headers, params=params)
        result = response.json()
        
        try:
            x = float(result['documents'][0]['x'])
            y = float(result['documents'][0]['y'])
            return x, y
        except IndexError:
            return None, None

    ## 도로명을 기반으로 좌표X와 좌표Y를 받아옴 (결측치에 한해서)
    def fill_missing_coords(row):
        if pd.isna(row['좌표X']) or pd.isna(row['좌표Y']):
            coords = roadname_to_coords.get(row['도로명'])
            if coords:
                return pd.Series(coords)
        return pd.Series([row['좌표X'], row['좌표Y']])
    
    roadname_to_coords = {}
    unique_roads = df.loc[df[['좌표X', '좌표Y']].isnull().any(axis=1), '도로명'].dropna().unique()

    api_key = '13b7b7a0b7a853100b56c56f19f6bc24'

    for road in tqdm(unique_roads) :
        x, y = get_coords_kakao(road, api_key)
        if x is not None and y is not None :
            roadname_to_coords[road] = (x, y)

    df[['좌표X', '좌표Y']] = df.apply(fill_missing_coords, axis=1)

    return df

In [2]:
train_data = Entire_Preprocessing(train_data)

NameError: name 'train_data' is not defined

In [37]:
train_data = train_data.dropna(subset=['좌표X', '좌표Y'])

In [38]:
kmeans = joblib.load('kmeans_model.pkl')

train_data['cluster'] = kmeans.predict(train_data[['좌표X', '좌표Y']])

/home/dst78/anaconda3/envs/Deep/lib/python3.11/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


In [39]:
train_data['좌표X'].isnull().sum()

0

# Bus & Subway

In [40]:
bus_data.head(3)

,노드 ID,정류소번호,정류소명,X좌표,Y좌표,정류소 타입
0,100000001,1001,종로2가사거리,126.987752,37.569808,중앙차로
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,중앙차로
2,100000003,1003,명륜3가.성대입구,126.998251,37.582581,중앙차로


In [41]:
subway_data.head(3)

,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868


In [42]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0

    lat1 = np.radians(lat1)[:, np.newaxis]
    lon1 = np.radians(lon1)[:, np.newaxis]
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c

In [43]:
apt_coords_rad = np.radians(train_data[['좌표Y', '좌표X']].to_numpy())
bus_coords_rad = np.radians(bus_data[['Y좌표', 'X좌표']].to_numpy())
subway_coords_rad = np.radians(subway_data[['위도', '경도']].to_numpy())

bus_tree = BallTree(bus_coords_rad, metric='haversine')
subway_tree = BallTree(subway_coords_rad, metric='haversine')

dist_bus_rad, _ = bus_tree.query(apt_coords_rad, k=1)
dist_sub_rad, _ = subway_tree.query(apt_coords_rad, k=1)

train_data['closest_bus'] = dist_bus_rad.flatten() * 6371.0
train_data['closest_sub'] = dist_sub_rad.flatten() * 6371.0

In [44]:
train_data.head(20)

,전용면적(㎡),건축년도,해제사유발생일,연면적,전용면적별세대현황(60㎡이하),전용면적별세대현황(60㎡~85㎡이하),전체세대수,주차대수,계약(연),계약(월),...,아파트명,등기신청일자,복도유형,"단지분류(아파트,주상복합등등)",도로명,target,세대당_주차대수,cluster,closest_bus,closest_sub
0,79.97,1987,0,22637.0,20.0,250.0,270.0,262.0,2017,12,...,개포6차우성,1,계단식,아파트,언주로 3,124000,0.970370,3,0.061783,1.129775
1,79.97,1987,0,22637.0,20.0,250.0,270.0,262.0,2017,12,...,개포6차우성,1,계단식,아파트,언주로 3,123500,0.970370,3,0.061783,1.129775
2,54.98,1987,0,22637.0,20.0,250.0,270.0,262.0,2017,12,...,개포6차우성,1,계단식,아파트,언주로 3,91500,0.970370,3,0.061783,1.129775
3,79.97,1987,0,22637.0,20.0,250.0,270.0,262.0,2018,1,...,개포6차우성,1,계단식,아파트,언주로 3,130000,0.970370,3,0.061783,1.129775
4,79.97,1987,0,22637.0,20.0,250.0,270.0,262.0,2018,1,...,개포6차우성,1,계단식,아파트,언주로 3,117000,0.970370,3,0.061783,1.129775
5,79.97,1987,0,22637.0,20.0,250.0,270.0,262.0,2018,1,...,개포6차우성,1,계단식,아파트,언주로 3,130000,0.970370,3,0.061783,1.129775
6,79.97,1987,0,22637.0,20.0,250.0,270.0,262.0,2018,3,...,개포6차우성,1,계단식,아파트,언주로 3,139500,0.970370,3,0.061783,1.129775
7,54.98,1987,0,22637.0,20.0,250.0,270.0,262.0,2018,4,...,개포6차우성,1,계단식,아파트,언주로 3,107500,0.970370,3,0.061783,1.129775
8,79.97,1987,0,22637.0,20.0,250.0,270.0,262.0,2018,6,...,개포6차우성,1,계단식,아파트,언주로 3,145000,0.970370,3,0.061783,1.129775
9,54.98,1987,0,22637.0,20.0,250.0,270.0,262.0,2018,7,...,개포6차우성,1,계단식,아파트,언주로 3,112000,0.970370,3,0.061783,1.129775


# Rate

금리의 경우 달 기준으로 금리가 변동되었기에, 웹 크롤링보다는 그냥 복사 및 붙여넣기를 이용하여 딕셔너리를 생성하였다.

또한, 계약년도의 고윳값을 확인했을 때, 2007년 이후이므로, 2007년 이전의 데이터는 가장 마지막이었던 2006년 8월을 제외하고는 전부 제거한다.

In [45]:
train_data['연_월'] = pd.PeriodIndex(year=train_data['계약(연)'], month=train_data['계약(월)'], freq='M')

In [46]:
train_data['계약(연)'].unique()

array([2017, 2018, 2011, 2012, 2008, 2009, 2023, 2022, 2021, 2020, 2019,
       2016, 2015, 2014, 2013, 2010, 2007])

In [47]:
rates = {
    "2025-02": 2.75,
    "2024-11": 3.00,
    "2024-10": 3.25,
    "2023-01": 3.50,
    "2022-11": 3.25,
    "2022-10": 3.00,
    "2022-08": 2.50,
    "2022-07": 2.25,
    "2022-05": 1.75,
    "2022-04": 1.50,
    "2022-01": 1.25,
    "2021-11": 1.00,
    "2021-08": 0.75,
    "2020-05": 0.50,
    "2020-03": 0.75,
    "2019-10": 1.25,
    "2019-07": 1.50,
    "2018-11": 1.75,
    "2017-11": 1.50,
    "2016-06": 1.25,
    "2015-06": 1.50,
    "2015-03": 1.75,
    "2014-10": 2.00,
    "2014-08": 2.25,
    "2013-05": 2.50,
    "2012-10": 2.75,
    "2012-07": 3.00,
    "2011-06": 3.25,
    "2011-03": 3.00,
    "2011-01": 2.75,
    "2010-11": 2.50,
    "2010-07": 2.25,
    "2009-02": 2.00,
    "2009-01": 2.50,
    "2008-12": 3.00,
    "2008-11": 4.00,
    "2008-10": 5.00,
    "2008-08": 5.25,
    "2007-08": 5.00,
    "2007-07": 4.75,
    "2006-08": 4.50
}

In [48]:
rate_changes = pd.DataFrame({
    '연_월' : list(rates.keys()),
    '금리' : list(rates.values())
})

rate_changes['연_월'] = pd.PeriodIndex(rate_changes['연_월'], freq='M')
rate_changes = rate_changes.sort_values('연_월').reset_index(drop=True)

rate_changes['start_month'] = rate_changes['연_월']
rate_changes['end_month'] = rate_changes['연_월'].shift(-1) - 1
rate_changes.at[rate_changes.index[-1], 'end_month'] = pd.Period('2099-12', freq='M')

def assign_rate(contract_period):
    matched = rate_changes[(rate_changes['start_month'] <= contract_period) & (rate_changes['end_month'] >= contract_period)]
    if not matched.empty:
        return matched.iloc[0]['금리']
    return None
    
train_data['금리'] = train_data['연_월'].apply(assign_rate)

In [49]:
train_data.to_csv('Add_rate_data.csv', index=False)